In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_deepseek import ChatDeepSeek

In [3]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")


In [4]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [5]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [6]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [8]:
raw_documents = TextLoader("tagged_description.txt", encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [9]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [10]:
import torch
from langchain.embeddings import HuggingFaceEmbeddings
device = "cuda" if torch.cuda.is_available() else "cpu"
# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2",
#     model_kwargs={"device": device}
# )

print(device)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-Mpnet-base-v2",
    model_kwargs={"device": device}
)

cuda


C:\Users\Jeyhun\AppData\Local\Temp\ipykernel_24784\3184804693.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [11]:
from langchain.vectorstores import Chroma
db_books = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory="./chroma_books",
)

In [12]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("İstifadə edilən cihaz:", device)
print("CUDA aktivdirmi?:", torch.cuda.is_available())
if device.type == 'cuda':
    print("GPU adı:", torch.cuda.get_device_name(0))
    print("GPU sayısı:", torch.cuda.device_count())


İstifadə edilən cihaz: cuda
CUDA aktivdirmi?: True
GPU adı: NVIDIA GeForce RTX 3060 Laptop GPU
GPU sayısı: 1


In [13]:
query = "II World War"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content="9780688085872 Despite the numerous books on World War II, until now there has been no one-volume survey that was both objective and comprehensive. Previous volumes have usually been written from an exclusively British or American point of view, or have ignored the important causes and consequences of the War. A Short History of World War II is essentially a military history, but it reaches from the peace settlements of World War I to the drastically altered postwar world of the late 1940's. Lucidly written and eminently readable, it is factual and accurate enough to satisfy professional historians. A Short History of World War II will appeal equally to the general reader, the veteran who fought in the War, and the student interested in understanding the contemporary political world."),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9780688085872 Despite the numerous books on World War II, unt

In [14]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3180,9780688085872,0688085873,A Short History of World War II,James L. Stokesbury,History,http://books.google.com/books/content?id=uDBhl...,"Despite the numerous books on World War II, un...",1980.0,3.93,416.0,454.0,A Short History of World War II,9780688085872 Despite the numerous books on Wo...


In [15]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = top_k)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [16]:
text = "The first battle"
retrieve_semantic_recommendations(text)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
999,9780195119206,0195119207,Ride of the Second Horseman,Robert L. O'Connell,History,http://books.google.com/books/content?id=1Xs7D...,"""Accurst be he that first invented war,"" wrote...",1997.0,4.23,320.0,12.0,Ride of the Second Horseman: The Birth and Dea...,"9780195119206 ""Accurst be he that first invent..."
1006,9780195168952,019516895X,Battle Cry of Freedom,James M. McPherson,History,http://books.google.com/books/content?id=09FkZ...,Filled with fresh interpretations and informat...,2005.0,4.34,867.0,22318.0,Battle Cry of Freedom: The Civil War Era,9780195168952 Filled with fresh interpretation...
1105,9780275942694,0275942694,The Heights of Courage,Avigdor Kahalani,History,http://books.google.com/books/content?id=7aIXJ...,"In October 1973, the State of Israel was invad...",1992.0,4.18,234.0,97.0,The Heights of Courage: A Tank Leader's War on...,"9780275942694 In October 1973, the State of Is..."
1526,9780345463050,0345463056,The Swarm War,Troy Denning,Fiction,http://books.google.com/books/content?id=9fPoc...,With a war between the Killiks and Chiss threa...,2006.0,3.49,357.0,3756.0,The Swarm War,9780345463050 With a war between the Killiks a...
1746,9780375829635,0375829636,Young Warriors,Tamora Pierce;Josepha Sherman,Juvenile Fiction,http://books.google.com/books/content?id=6EnZw...,Fifteen original short stories by various auth...,2005.0,3.91,312.0,3583.0,Young Warriors: Stories of Strength,9780375829635 Fifteen original short stories b...
1840,9780385423755,0385423756,On the Origins of War and the Preservation of ...,Donald Kagan,History,http://books.google.com/books/content?id=dVMsA...,Analyzes history to determine why there are wa...,1996.0,4.01,573.0,494.0,On the Origins of War and the Preservation of ...,9780385423755 Analyzes history to determine wh...
1959,9780394746234,0394746236,"The Civil War, a Narrative: Fort Sumter to Per...",Shelby Foote,History,http://books.google.com/books/content?id=B9d3A...,Follows the course of the war from 1862 to 186...,1986.0,4.42,856.0,9457.0,"The Civil War, a Narrative: Fort Sumter to Per...",9780394746234 Follows the course of the war fr...
2237,9780441014224,0441014224,The God in the Moon,Richard A. Knaak,Fiction,http://books.google.com/books/content?id=ussPy...,Nermesa Klandes joins the Aquilonian army in h...,2006.0,3.72,297.0,57.0,The God in the Moon,9780441014224 Nermesa Klandes joins the Aquilo...
2459,9780465024964,0465024963,The Classical World,Robin Lane Fox,History,http://books.google.com/books/content?id=Srhcj...,"Armies and empires, statesmen and tyrants--the...",2005.0,3.92,672.0,1222.0,The Classical World: An Epic History from Home...,"9780465024964 Armies and empires, statesmen an..."
2721,9780553383683,055338368X,Gates Of Fire,Steven Pressfield,Fiction,http://books.google.com/books/content?id=wdKJD...,Chronicles the battle of three hundred Spartan...,2005.0,4.41,392.0,20649.0,Gates Of Fire: An Epic Novel of the Battle of ...,9780553383683 Chronicles the battle of three h...
